In [2]:
import matplotlib.pyplot as plt
import numpy as np
import json

In [3]:
indices = ['hnsw', 'vamana', 'hcnng', 'nsg']
colors = ['tomato', 'deepskyblue', 'forestgreen', 'PeachPuff']
rhoes = [2.0**(0), 2**(-1), 2**(-2), 2**(-3), 2**(-4)]
rho_map = {
  2**(0): r'$2^0$',
  2**(-1): r'$2^{-1}$',
  2**(-2): r'$2^{-2}$',
  2**(-3): r'$2^{-3}$',
  2**(-4): r'$2^{-4}$'
}
dataset = 'sift-128-euclidean'

In [ ]:
from function import pareto_frontier
# from matplotlib.ticker import ScalarFormatter

nrows = 1
ncols = len(rhoes)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 1.5))

# for i in range(nrows):
axes[0].set_ylabel('QPS', fontsize=14)
# for i in range(num_datasets):
for j in range(len(rhoes)):
  rho = rhoes[j]
  # ds = 'sift-128-euclidean'
  for k in range(len(indices)):
    index = indices[k]
    recall_limit = 0.5
    recall_gap = 0.1
    with open(f"perf/factor/rho/pf.{index}.{dataset}.uniform-0.0-1.0.uniform-0.0-{rho}.k10.json", 'r') as file:
      baseline = pareto_frontier(json.load(file), recall_limit)
    with open(f"perf/factor/rho/qt.{index}.{dataset}.uniform-0.0-1.0.uniform-0.0-{rho}.k10.json", 'r') as file:
      method = pareto_frontier(json.load(file), recall_limit)
    # baseline = [(item['qps'], item['recall']) for item in baseline if item['recall'] > recall_limit]
    # method = [(item['qps'], item['recall']) for item in method if item['recall'] > recall_limit]
    axes[j].plot([item['recall'] for item in baseline], 
                                [item['qps'] for item in baseline], marker='s', 
                                  color=colors[k], linewidth=1.2, linestyle='dashed', markersize=3)
    axes[j].plot([item['recall'] for item in method], 
                                [item['qps'] for item in method], marker='o', 
                                  color=colors[k], linewidth=1.2, markersize=3)
  # if i == nrows - 1:
  axes[j].set_xlabel(f'Recall@10', fontsize=14)
  axes[j].set_xticks(np.arange(recall_limit, 1.001, recall_gap))
  axes[j].set_yscale("log", base=10)
  # axes[j].yaxis.set_major_formatter(ScalarFormatter())
  # axex[j].tick_params(axis='y', labelsize=10)
  # axes[j].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
  axes[j].grid(True)
  axes[j].set_title(f"SIFT1M ({r'$\rho=$' + f"{rho_map[rho]}"})",fontsize=14)

leg = list()
for idx in indices:
  if idx != 'vamana':
    leg += [idx.upper(), 'Hi-PNG-'+(idx).upper()]
  else:
    leg += ['Vamana', ('Hi-PNG-Vamana')]

fig.subplots_adjust(wspace=0.5, hspace=0.2)
fig.legend(leg, loc='center', bbox_to_anchor=(0.51, 1.2), ncol=8, frameon=False, fontsize=14)
fig.savefig(f'figure/factor/rho.pdf', dpi=300, bbox_inches='tight', pad_inches=0)
fig.show()